In [12]:
from bs4 import BeautifulSoup
import pandas as pd
from rapidfuzz import process
from rapidfuzz import fuzz
from glob import glob
import warnings
from bs4 import XMLParsedAsHTMLWarning
warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)
filepath  = "export_job_15918348/**/*.xml" 
files = glob(filepath, recursive=True)
import time
import numpy as np
import re

In [13]:
# files = [file for file in files if file.split('\\')[-1].replace('.xml','') in ["PlakkaatboekDeel03", "PlakkaatboekDeel04", "PlakkaatboekDeel13", "PlakkaatboekDeel14"]]

# 16 ver and 15 ver uses articles, 15th uses old month names 

In [14]:
maanden = [
    "louwmaand", "sprokkelmaand", "lentmaand", "grasmaand", "blooimand", "zomermaand", 
    "hooimaand", "oogstmaand", "herfstmaand", "wijnmaand", "slaghtmaand", "wintermaand",
    "januari", "februari", "maart", "april", "mei", "juni",
    "juli", "augustus", "september", "oktober", "november", "december",
    "Januari", "Februari", "Maart", "April", "Mei", "Juni",
    "Juli", "Augustus", "September", "Oktober", "November", "December",
    'Louwmaand', 'Sprokkelmaand', 'Lentmaand', 'Grasmaand', 'Blooimand', 'Zomermaand',
    'Hooimaand', 'Oogstmaand', 'Herfstmaand', 'Wijnmaand', 'Slaghtmaand', 'Wintermaand'
]

def fuzzy_matching(word, threshold=80):
    match1, score1, _ = process.extractOne(word, maanden, scorer=fuzz.ratio)
    return score1 >= threshold and len(word) >= 3

def fuzzy_matching_google(word, threshold=70):
        match, score, _ = process.extractOne(word, ['Digitized by Google'])
        return score >= threshold or "google" in word.lower()

In [15]:
max([len(i) for i in maanden])

13

### GET LINES

In [16]:
def format_lines(test_df):
    bool_has_year = []
    bool_has_date = []
    bool_has_google = []
    for x in test_df['line']:
        splits = x.split('.')
        # split at either first number or first space: need to find month
        dates = list(filter(None, re.split('(\d{1,})|\s', splits[0])))
        # print(dates)
    
        if 0 < len(dates):
            # print(dates[-1], fuzzy_matching(dates[-1]))
            if fuzzy_matching(dates[-1]): # To do when merging check if random dates arent picked up.
                bool_has_date.append(True)          # if date then no year
                bool_has_year.append(False)
            else:
                bool_has_date.append(False)         # if no date then check for year
                bool_has_year.append(dates[0].isdigit() and len(dates[0])==4)
        else:
            bool_has_year.append(False)             # otherwise just text
            bool_has_date.append(False)
        
        bool_has_google.append(fuzzy_matching_google(x))
        
    test_df['boolHasLegisDate'] = bool_has_date
    test_df['boolHasLegisYear'] = bool_has_year
    test_df['boolHasGoogle'] = bool_has_google
    test_df = test_df[test_df['boolHasGoogle']==False]
    return test_df

In [17]:
# first get all features
for file in files:

    df = None

    with open(file, encoding='UTF-8') as fp:
        soup = BeautifulSoup(fp, 'xml', from_encoding='utf-8')
    num_pages = len(soup.find_all("pb"))
    filename = file.split('\\')[-1].replace('.xml','')
    print(f"On xml: {filename}")

    start = time.time()

    # line wise  
    ps = soup.find_all("zone", attrs={"rendition": 'Line'})
    ids = ["#" + y.attrs['xml:id'] for y in ps]
    points = [[int(i) for x in y.attrs["points"].split(' ') for i in x.split(',')] for y in ps]
    # print(len(points), len(ids))

    # first df for points and ids
    df1 = pd.DataFrame({"points": points, "id": ids})

    ps = soup.find_all("l")
    texts = [x.text for x in ps]
    ids = [x.attrs['facs'] for x in ps]
    # print(len(texts), len(ids))

    # second df for ids and lines
    df2 = pd.DataFrame({"line": texts, "id": ids})

    # since id is 1->1 merge on id
    df = pd.merge(df1, df2, on='id')
    # remove nans and empty strs
    df.dropna(inplace=True)
    df = df[df['line'].str.strip().astype(bool)]
    df = format_lines(df)
    end = time.time()
    print(f"     Time: {end-start}")
    df.to_csv(f'raw/placaatboek/csvs/{filename}_lines.csv', encoding="utf-8")    

On xml: PlakkaatboekDeel03
     Time: 0.9115850925445557
On xml: PlakkaatboekDeel01
     Time: 1.3408699035644531
On xml: PlakkaatboekDeel02
     Time: 0.8468208312988281
On xml: PlakkaatboekDeel05
     Time: 1.0240871906280518
On xml: PlakkaatboekDeel17
     Time: 1.505370855331421
On xml: PlakkaatboekDeel16
     Time: 1.0571069717407227
On xml: PlakkaatboekDeel15
     Time: 1.5350995063781738
On xml: PlakkaatboekDeel09
     Time: 0.8653302192687988
On xml: PlakkaatboekDeel04
     Time: 1.2274816036224365
On xml: PlakkaatboekDeel14
     Time: 1.1546740531921387
On xml: PlakkaatboekDeel06
     Time: 1.1519150733947754
On xml: PlakkaatboekDeel08
     Time: 1.321084976196289
On xml: PlakkaatboekDeel13
     Time: 1.029449701309204
On xml: PlakkaatboekDeel11
     Time: 1.1968235969543457
On xml: PlakkaatboekDeel07
     Time: 1.1547691822052002
On xml: PlakkaatboekDeel10
     Time: 1.3759105205535889
On xml: PlakkaatboekDeel12
     Time: 1.476513385772705


### Classifier for detecting year

In [18]:
# import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
# import pandas as pd
# from sklearn.neighbors import KNeighborsClassifier
# import pickle

In [19]:
# test_df= 

In [20]:
# remove_coords = pd.DataFrame(test_df[test_df['boolHasLegisYear']==1]['points'])
# remove_coords['boolRemove'] = [1]*len(remove_coords)
# non_remove_coords = pd.DataFrame(test_df[test_df['boolHasLegisYear']==0].sample(len(remove_coords)*3)['points'])
# non_remove_coords['boolRemove'] = [0]*len(non_remove_coords)
# classifier_dataset = pd.concat([remove_coords, non_remove_coords])
# classifier_dataset = classifier_dataset.sample(frac=1).reset_index(drop=True)

In [21]:
# classifier_dataset

In [22]:
# X = pd.DataFrame(classifier_dataset['points'].to_list()).fillna(0)
# 
# y = classifier_dataset['boolRemove'].to_numpy()
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
# 
# clf = KNeighborsClassifier(n_neighbors=3)
# clf.fit(X_train, y_train)
# predictions = clf.predict(X_test)
# cm = confusion_matrix(y_test, predictions, labels=clf.classes_, normalize='pred')
# disp = ConfusionMatrixDisplay(confusion_matrix=cm,
#                               display_labels=clf.classes_)
# disp.plot()
# plt.savefig("C:/Users/imruh/Documents/date-extraction-dutch-east-indies/data/CM_headingremoval.png")
# pickle.dump(clf, open('C:/Users/imruh/Documents/date-extraction-dutch-east-indies/data/KNN_headingremoval', 'wb'))